### 0 Getting information about P from file "P_info_n.txt"

Setting n:

In [ ]:
n=4
round_balls=True

In [ ]:
import ast #helps to change string into list below

P_info is the text-file generated by the code 'Generating balls in P'. It contains a list of two elements. The first being 147 (the number of elements in the ball of which we are looking for units). The other element of the list is in itself a list of lists. Each sublist containing the products that are in fact equal - noting that the products are repesented by numbers from 2*147+1 and 147^2 up from there.

In [ ]:
f = open(f"P_info_{n}.txt","r")
p_info=ast.literal_eval(f.read())

In [ ]:
p_info[0]

### 1 Starting the list of clauses

Each literal is represented by a positive integer and a clause is simply a list of positive integers.

In [ ]:
clauses=[]

In [ ]:
if not round_balls:
    #Assuming a_1 is nonzero
    clauses.append([1])
    #Assuming that one of the other elements has support - i.e it is non-trivial
    clauses.append(list(range(2,p_info[0]+1)))

In [ ]:
if round_balls:
    #Assuming that at least one coef is nonzero (assuming the element is not zero)
    clauses.append(list(range(1,p_info[0]+1)))
    #Assuming that there is not precisely 1 element in the support (trivial)
    for i in range(1,p_info[0]+1):
        clauses.append(list(range(1,p_info[0]+1)))
        clauses[i][i-1]=-clauses[i][i-1] 

In [ ]:
clauses

### 2 Storing products of all pairs of coefficients

Now, we want the literals from 2n to 2n+n*n to store the value of all possible products of coefficients. 'True' corresponding to the value 1 and 'False' corresponding to the value 0.

In [ ]:
#We define the fist free literal - in this way we store auxilirary variables in the next vacant number
first_free_literal=2*p_info[0]+1
for x in range(1,p_info[0]+1):
    for y in range(p_info[0]+1,2*p_info[0]+1):
        clauses.append([x,-first_free_literal])
        clauses.append([y,-first_free_literal])
        clauses.append([-x,-y,first_free_literal])
        first_free_literal=first_free_literal+1

### 3 Ensuring that the sums of products that reduce to the same element add to 1 or 0

In [ ]:
#We will continue to add auxilirary variables by using the stored first_free_literal
first_free_literal

In [ ]:
def sum_to_zero(lst):
    ''' This function adds clauses to the system enforcing the sum of variables in a given list to be 0. 
        If the list is of length up to 3 it does this straight away. Otherwise it returns the list to long_sum.'''
    
    if len(lst)==1:
        clauses.append([-lst[0]]) #Enforcing this single value to be 0

    if len(lst)==2:
        x,y=lst[0],lst[1]
        clauses.append([x,-y]) #Enforcing the sum to be 0 (ruling out the 2 cases that add to 1)
        clauses.append([y,-x])

    if len(lst)==3:
        x,y,z=lst[0],lst[1],lst[2]
        clauses.append([-x,-y,-z]) #Ruling out the four cases that add to 1.
        clauses.append([-x,y,z])
        clauses.append([x,-y,z])
        clauses.append([x,y,-z])
        
    if len(lst)>3: #Sending to other function
        return(long_sum(lst,sum_to_zero))

    return None

In [ ]:
def long_sum(lst,fnct):
    ''' This function makes an auxilirary variable, holding the sum of the first three variables in the list.
        The first 3 variables are removed and the new variable is saved in the list and the list is the returned to fnct.
        To keep track of the next free literal, the function adds 1 to the global variable first_free_literal.'''
    
    #Loading and changing the global value of the first free literal
    global first_free_literal
    a=first_free_literal
    first_free_literal=first_free_literal+1

    #Adding clauses about a and the first 3 elem in list
    x,y,z=lst[0],lst[1],lst[2]
    clauses.append([-x,-y,-z,a])
    clauses.append([-x,y,z,a])
    clauses.append([x,-y,z,a])
    clauses.append([x,y,-z,a])
    clauses.append([x,y,z,-a])
    clauses.append([-x,-y,z,-a])
    clauses.append([x,-y,-z,-a])
    clauses.append([-x,y,-z,-a])
    
    #Alterating list before returning to sum_to_zero
    new_lst=lst[3:]
    new_lst.append(a)
    
    return(fnct(new_lst))

In [ ]:
def sum_to_one(lst):
    ''' This function adds clauses to the system enforcing the sum of variables in a given list to be 1. 
        If the list is of length up to 3 it does this straight away. Otherwise it returns the list to long_sum.'''
    
    if len(lst)==1:
        clauses.append([lst[0]]) #Enforcing this single value to be 1
        
    if len(lst)==2:
        x,y=lst[0],lst[1]
        clauses.append([x,y]) #Enforcing this single value to be 1 (leftover combinations of the sum to zero)
        clauses.append([-y,-x])

    if len(lst)==3:
        x,y,z=lst[0],lst[1],lst[2]
        clauses.append([x,y,z]) #Enforcing this single value to be 1 (leftover combinations of the sum to zero)
        clauses.append([x,-y,-z])
        clauses.append([-x,y,-z])
        clauses.append([-x,-y,z])
        
    if len(lst)>3: #Sending to other function
        return(long_sum(lst,sum_to_one))

    return None

In [ ]:
#Now running the functions on the imported list of lists
sum_to_one(p_info[1][0])
for i in p_info[1][1:]:
    sum_to_zero(i)

### Looking at and exporting clauses

Here is the number of litterals (+1) and number of clauses

In [ ]:
print(first_free_literal)
print(len(clauses))

In [ ]:
if not round_balls:
    with open(f'clauses_{n}.txt','w') as g:
        g.write(str(clauses))

In [ ]:
if round_balls:
    with open(f'clauses_round_{n}.txt','w') as g:
        g.write(str(clauses))